# Testing GCP scripting

## Setup

In [1]:
import os
import datetime
import random
from google.cloud import storage
from google.cloud import secretmanager
from dotenv import load_dotenv
load_dotenv()


True

## Data refresh cloud function

In [2]:
class MockRequest:
    def __init__(self, args=None, json_body=None):
        self.args = args or {}
        self.json = json_body or {}

In [3]:
# Function to retrieve secrets based on the environment
def get_secret(secret_name):
    if os.environ.get('GCP_ENVIRONMENT'):
        # Access secret from GCP Secret Manager
        client = secretmanager.SecretManagerServiceClient()
        project_id = os.environ.get('GCP_PROJECT_ID')
        name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
        response = client.access_secret_version(request={"name": name})
        secret_value = response.payload.data.decode("UTF-8")
    else:
        # Access secret from local environment variable
        secret_value = os.environ.get(secret_name, 'VALUE_NOT_SET')
    return secret_value

# Function to generate fake data
def generate_fake_data():
    date = datetime.datetime.now().strftime('%Y-%m-%d')
    weight = random.uniform(170, 180)  # Random float between 170 and 180
    calories = random.uniform(1500, 3000)  # Random float between 1500 and 3500
    steps = random.uniform(5000, 15000)  # Random float between 5000 and 15000

    # Format as CSV line
    data_line = f'{date},{weight:.2f},{calories:.2f},{steps:.2f}\n'
    return data_line


In [4]:
get_secret('CLOUD_STORAGE_BUCKET')
# generate_fake_data()

'weight-forecaster-poc-data'

In [5]:
# Function to append data to a file in Cloud Storage
def append_to_storage(bucket_name, file_name, data_line):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Check if the file exists and download its contents
    if blob.exists():
        current_content = blob.download_as_text()
    else:
        current_content = ''

    # Append the new line of data
    new_content = current_content + data_line

    # Upload the updated content back to the Cloud Storage file
    blob.upload_from_string(new_content)

def data_refresh(request):
    # Retrieve the Cloud Storage bucket name from secrets
    bucket_name = get_secret('CLOUD_STORAGE_BUCKET')
    file_name = 'fitness_data.csv'

    data_line = generate_fake_data()
    append_to_storage(bucket_name, file_name, data_line)

    return f'Appended new data to {file_name} in bucket {bucket_name}'

In [6]:
mock_request = MockRequest()
data_refresh(mock_request)

'Appended new data to fitness_data.csv in bucket weight-forecaster-poc-data'

TODOs:
- test locally as a script (instead of in a jupyter notebook)
- test running in the GCP environment...not sure if there is an intermediate step or we just try to deploy it as a cloud function
- how do we manage the cloud function? if I update the script, can I commit, then push it to github, and then have GCP get the updated version? Do we use a docker container at this stage?
